# Test 2: Model-Observation comparison with ROMS and CrocoLake

The goal of this test is to learn how to generate your own [obs_seq files](https://docs.dart.ucar.edu/en/latest/guide/detailed-structure-obs-seq.html) from CrocoLake and learn about another interactive widget: the interactive profile. We will select a specific region, interpolate our model output to the temperature and salinity values of CrocoLake data in that region, and compare the interpolated model and the float's profiles.

## Generating your own obs_seq.in files

In [1]:
# crocolake_path = '$CROCOLAKE_PATH/0007_PHY_CROCOLAKE-QC-MERGED'

# import os
# basename = "myCL_obs_seq_"
# outdir = "/home/enrico/myWHOI/CrocoLake/CrocoCamp/roms_tests/input/test_2/in_CrocoLake/"
# basename = os.path.expandvars(outdir+basename)
# outdir = os.path.expandvars(outdir)
# crocolake_path = os.path.expandvars(crocolake_path)

In [2]:
# import datetime
# from convert_crocolake_obs import ObsSequence

# # define horizontal region
# LAT0 = 27
# LAT1 = 52
# LON0 = -82
# LON1 = -35

# # define variables to import from CrocoLake
# selected_variables = [
#     "DB_NAME",  # ARGO, GLODAP, SprayGliders, OleanderXBT, Saildrones
#     "JULD", # this contains timestamp
#     "LATITUDE",
#     "LONGITUDE",
#     "PRES", # This will be automatically converted to depths in meters
#     "TEMP",
#     "PRES_QC",
#     "TEMP_QC",
#     "PRES_ERROR",
#     "TEMP_ERROR",
#     "PSAL",
#     "PSAL_QC",
#     "PSAL_ERROR"
# ]

# # month and year are constant in out case
# year0 = 2018
# month0 = 5
# day0 = 31
# basename = basename + f"ROMS_test2"
# if not os.path.exists(outdir):
#     os.makedirs(outdir, exist_ok=True)

# # we loop to generate one file per day
# for j in range(5):

#     # set date range
#     day1 = day0+j
#     day2 = day1+1

#     month1 = month0
#     month2 = month0

#     if day1>31:
#         day1 = day1-31
#         month1 = month1+1
#     if day2>31:
#         day2 = day2-31
#         month2 = month2+1
        
#     date0 = datetime.datetime(year0, month1, day1, 0, 0, 0)
#     date1 = datetime.datetime(year0, month2, day2, 0, 0, 0)
#     print(f"Converting obs between {date0} and {date1}")

#     # this defines AND filters, i.e. we want to load each observation that has latitude within the given range AND longitude within the given range, etc.
#     # to exclude NaNs, impose a range to a variable
#     and_filters = (
#         ("LATITUDE",'>',LAT0),  ("LATITUDE",'<',LAT1),
#         ("LONGITUDE",'>',LON0), ("LONGITUDE",'<',LON1),
#         ("PRES",'>',-1e30), ("PRES",'<',1e30),
#         ("JULD",">",date0), ("JULD","<",date1)
#     )

#     # this adds OR conditions to the and_filters, i.e. we want to load all observations that statisfy the AND conditions above, AND that have finite salinity OR temperature values
#     db_filters = [
#         list(and_filters) + [("PSAL", ">", -1e30), ("PSAL", "<", 1e30)],
#         list(and_filters) + [("TEMP", ">", -1e30), ("TEMP", "<", 1e30)],
#     ]

#     # generate output filename
#     obs_seq_out = basename + f".{year0:04d}{month1:02d}{day1:02d}.out"

#     # generate obs_seq.in file
#     obsSeq = ObsSequence(
#         crocolake_path,
#         selected_variables,
#         db_filters,
#         obs_seq_out=obs_seq_out,
#         loose=True
#     )
#     obsSeq.write_obs_seq()


## Running the workflow

In [6]:
from crococamp.workflows import WorkflowModelObs

# interpolate model onto obs space for the single float
workflow_float = WorkflowModelObs.from_config_file('config_roms_test2.yaml')
workflow_float.run(clear_output=True,trim_obs=False) #use flag clear_output=True if you want to re-run it and automatically clean all previous output


Clearing all output folders...
  Clearing folder: /home/enrico/myWHOI/CrocoLake/CrocoCamp/roms_tests/output/test_2/out_parquet
  Clearing folder: /home/enrico/myWHOI/CrocoLake/CrocoCamp/roms_tests/output/test_2/input_bckp
  Deleted file: /home/enrico/myWHOI/CrocoLake/CrocoCamp/roms_tests/output/test_2/input_bckp/input.nml_0000.backup
  Clearing folder: /home/enrico/myWHOI/CrocoLake/CrocoCamp/roms_testsoutput/test_2/out_trimmed_obs_seq_in
  Clearing folder: /home/enrico/myWHOI/CrocoLake/CrocoCamp/roms_tests/output/test_2/out_obs_seq_in
  All output folders cleared.

Starting files processing.
  Initializing input.nml for process_model_obs subprocess...
    Setting up symlink for input.nml...
    Created backup: input.nml.backup
ocean model: ROMS_rutgers
  Processing observation types from config...
    Target observation types: ['ADCP_SALINITY', 'ADCP_TEMPERATURE', 'APB_SALINITY', 'APB_TEMPERATURE', 'ARGO_SALINITY', 'ARGO_TEMPERATURE', 'BOTTLE_SALINITY', 'BOTTLE_TEMPERATURE', 'CTD_SALIN

AttributeError: 'NoneType' object has no attribute 'read'

## Displaying the interactive map

We can load the data with the same tools as in Tutorial 1. Note that in this case all interpolations were succesfull, so `get_good_model_obs_df()` and `get_all_model_obs()` will return the same dataframe (`get_failed_model_obs()` will return an empty dataframe).

In [2]:
good_model_obs_df = workflow_float.get_good_model_obs_df(compute=True) # compute=True triggers the compute of the dask dataframe, returning a pandas dataframe with data loaded in memory
good_model_obs_df.head()                                                   # displays first 5 rows in the dataframe

,time,longitude,latitude,vertical,type,interpolated_model,obs,obs_err_var,interpolated_model_QC,difference,abs_difference,squared_difference,normalized_difference,log_likelihood
4,2004-02-01 16:27:38,318.040,31.605,171.430778,ARGO_SALINITY,36.502508,0.036544,0.02,1.0,-36.465964,36.465964,1329.766516,-257.853306,-33243.126579
8,2004-02-01 11:56:48,322.099,41.942,321.010774,ARGO_SALINITY,35.628180,0.035789,0.01,1.0,-35.592391,35.592391,1266.818311,-355.923916,-63339.533344
9,2004-02-01 16:27:38,318.040,31.605,1871.497324,ARGO_SALINITY,35.054915,0.035033,0.02,1.0,-35.019882,35.019882,1226.392152,-247.627965,-30658.767412
12,2004-02-01 11:56:48,322.099,41.942,1038.551600,ARGO_SALINITY,35.074258,0.035012,0.01,1.0,-35.039246,35.039246,1227.748768,-350.392465,-61386.056101
25,2004-02-01 11:56:48,322.099,41.942,594.040350,ARGO_SALINITY,35.085573,0.035093,0.01,1.0,-35.050480,35.050480,1228.536155,-350.504805,-61425.425468


Now load the interactive map as in Tutorial 1. What do you see? (Again, you can try and code it yourself or load the solution and run it.)

In [3]:
good_model_obs_df

,time,longitude,latitude,vertical,type,interpolated_model,obs,obs_err_var,interpolated_model_QC,difference,abs_difference,squared_difference,normalized_difference,log_likelihood
4,2004-02-01 16:27:38,318.040,31.605,171.430778,ARGO_SALINITY,36.502508,0.036544,0.020,1.0,-36.465964,36.465964,1329.766516,-257.853306,-33243.126579
8,2004-02-01 11:56:48,322.099,41.942,321.010774,ARGO_SALINITY,35.628180,0.035789,0.010,1.0,-35.592391,35.592391,1266.818311,-355.923916,-63339.533344
9,2004-02-01 16:27:38,318.040,31.605,1871.497324,ARGO_SALINITY,35.054915,0.035033,0.020,1.0,-35.019882,35.019882,1226.392152,-247.627965,-30658.767412
12,2004-02-01 11:56:48,322.099,41.942,1038.551600,ARGO_SALINITY,35.074258,0.035012,0.010,1.0,-35.039246,35.039246,1227.748768,-350.392465,-61386.056101
25,2004-02-01 11:56:48,322.099,41.942,594.040350,ARGO_SALINITY,35.085573,0.035093,0.010,1.0,-35.050480,35.050480,1228.536155,-350.504805,-61425.425468
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,2004-02-01 16:27:38,318.040,31.605,216.473346,ARGO_SALINITY,36.421886,0.036492,0.020,1.0,-36.385393,36.385393,1323.896860,-257.283588,-33096.385169
777,2004-02-01 01:27:31,320.516,36.441,1067.411597,ARGO_SALINITY,35.204167,0.035170,0.005,1.0,-35.168998,35.168998,1236.858388,-497.364739,-123684.111339
786,2004-02-01 01:27:31,320.516,36.441,97.455613,ARGO_SALINITY,36.273679,0.036389,0.005,1.0,-36.237290,36.237290,1313.141189,-512.472676,-131312.391596
791,2004-02-01 11:56:48,322.099,41.942,741.690372,ARGO_SALINITY,35.121215,0.035043,0.010,1.0,-35.086172,35.086172,1231.039455,-350.861722,-61550.590477


In [4]:
from crococamp.viz import InteractiveWidgetMap

# Create an interactive map widget to visualize model-observation comparisons
# The widget provides controls for selecting variables, observation types, and time ranges
widget = InteractiveWidgetMap(good_model_obs_df)
widget.setup()


The previous cell should show a dot in the middle of the ocean, which is not very informative. We can then use `MapConfig` to pass extra arguments to `InteractiveWidgetMap`, among which the extent of the map area to plot. Play with the `map_extent` values in the cell below and re-execute the cell until you're happy with your plotted region.

In [5]:
from crococamp.viz import MapConfig

map_config = MapConfig(
    map_extent=(-90,0,0,90) #(lon_min, lon_max, lat_min, lat_max)
)

# Create an interactive map widget to visualize model-observation comparisons
# The widget provides controls for selecting variables, observation types, and time ranges
widget = InteractiveWidgetMap(good_model_obs_df, config=map_config)
widget.setup()

### Interactive profile

Finally, we can load the interactive profile widget to explore how our model performed compared to the specific float measurements we selected:

In [ ]:
from crococamp.viz import InteractiveWidgetProfile
# Create an interactive profile widget to analyze vertical profiles
# This is ideal for analyzing single float/CTD profiles vs model data
widget = InteractiveWidgetProfile(good_model_obs_df)
widget.setup()

You can also pass some settings through the `config` argument of the InteractiveWidgetProfile class:

In [ ]:
from crococamp.viz import ProfileConfig

# Customize the profile widget appearance and behavior
profile_config = ProfileConfig(
    figure_size=(7, 7),
    marker_size=5,
    marker_alpha=0.6,
    invert_yaxis=False,  # Don't invert for this example
    grid=True
)

# Create widget with custom configuration
widget = InteractiveWidgetProfile(good_model_obs_df, x='obs', y='interpolated_model', config=profile_config)
widget.setup()